In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns',None)
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [2]:
train_df = pd.read_csv('train.csv')

In [4]:
test_df = pd.read_csv('test.csv')

In [5]:
train_df

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [6]:
test_df

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...
...,...,...,...
8984,29957,Supporting mixed-datatype matrix multiplicatio...,We approach the problem of implementing mixe...
8985,29958,An axiomatic basis for Blackwell optimality,In the theory of Markov decision processes (...
8986,29959,GeneVis - An interactive visualization tool fo...,GeneVis is a web-based tool to visualize com...
8987,29960,Quantifying the causal effect of speed cameras...,This paper quantifies the effect of speed ca...


In [7]:
train_df.drop('ID',axis=1,inplace=True)
test_df.drop('ID',axis=1,inplace=True)

In [8]:
target_features = list(train_df.columns[2:8])

In [9]:
train_df['word_count_ABSTRACT'] = train_df['ABSTRACT'].apply(lambda x: len(str(x).split(" "))) 
train_df['word_count_TITLE'] = train_df['TITLE'].apply(lambda x: len(str(x).split(" "))) 

test_df['word_count_ABSTRACT'] = test_df['ABSTRACT'].apply(lambda x: len(str(x).split(" "))) 
test_df['word_count_TITLE'] = test_df['TITLE'].apply(lambda x: len(str(x).split(" "))) 

In [10]:
def avg_word(sentence):
    words = sentence.split()    
    return (sum(len(word) for word in words)/len(words))

In [11]:
train_df['avg_word_ABSTRACT'] = train_df['ABSTRACT'].apply(lambda x: avg_word(x))
train_df['avg_word_TITLE'] = train_df['TITLE'].apply(lambda x: avg_word(x))

test_df['avg_word_ABSTRACT'] = test_df['ABSTRACT'].apply(lambda x: avg_word(x))
test_df['avg_word_TITLE'] = test_df['TITLE'].apply(lambda x: avg_word(x))

In [12]:
train_df['stopwords_ABSTRACT'] = train_df['ABSTRACT'].apply(lambda x: len([x for x in x.split() if x in stop]))
train_df['stopwords_TITLE'] = train_df['TITLE'].apply(lambda x: len([x for x in x.split() if x in stop]))

test_df['stopwords_ABSTRACT'] = test_df['ABSTRACT'].apply(lambda x: len([x for x in x.split() if x in stop]))
test_df['stopwords_TITLE'] = test_df['TITLE'].apply(lambda x: len([x for x in x.split() if x in stop]))

In [13]:
train_df['numerics_ABSTRACT'] = train_df['ABSTRACT'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
train_df['numerics_TITLE'] = train_df['TITLE'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

test_df['numerics_ABSTRACT'] = test_df['ABSTRACT'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
test_df['numerics_TITLE'] = test_df['TITLE'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

In [14]:
train_df['ABSTRACT'] = train_df['ABSTRACT'].str.replace('[^\w\s]','')
train_df['TITLE'] = train_df['TITLE'].str.replace('[^\w\s]','')

test_df['ABSTRACT'] = test_df['ABSTRACT'].str.replace('[^\w\s]','')
test_df['TITLE'] = test_df['TITLE'].str.replace('[^\w\s]','')

In [15]:
train_df['non_alphanumerics_ABSTRACT'] = train_df['ABSTRACT'].apply(lambda x: len([x for x in x.split() if not x.isalnum()]))
train_df['non_alphanumerics_TITLE'] = train_df['TITLE'].apply(lambda x: len([x for x in x.split() if not x.isalnum()]))

test_df['non_alphanumerics_ABSTRACT'] = test_df['ABSTRACT'].apply(lambda x: len([x for x in x.split() if not x.isalnum()]))
test_df['non_alphanumerics_TITLE'] = test_df['TITLE'].apply(lambda x: len([x for x in x.split() if not x.isalnum()]))

In [16]:
train_df['upper_case_ABSTRACT'] = train_df['ABSTRACT'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
train_df['upper_case_TITLE'] = train_df['TITLE'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

test_df['upper_case_ABSTRACT'] = test_df['ABSTRACT'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
test_df['upper_case_TITLE'] = test_df['TITLE'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [17]:
train_df['ABSTRACT'] = train_df['ABSTRACT'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train_df['TITLE'] = train_df['TITLE'].apply(lambda x: " ".join(x.lower() for x in x.split()))

test_df['ABSTRACT'] = test_df['ABSTRACT'].apply(lambda x: " ".join(x.lower() for x in x.split()))
test_df['TITLE'] = test_df['TITLE'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [18]:
train_df['ABSTRACT'] = train_df['ABSTRACT'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train_df['TITLE'] = train_df['TITLE'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

test_df['ABSTRACT'] = test_df['ABSTRACT'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test_df['TITLE'] = test_df['TITLE'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [19]:
from nltk.stem import WordNetLemmatizer,PorterStemmer
st = PorterStemmer()
lm = WordNetLemmatizer()

train_df['ABSTRACT'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
train_df['TITLE'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

test_df['ABSTRACT'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
test_df['TITLE'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0       closedform margin likelihood gammapoisson matr...
1       laboratori midir spectra equilibr igneou meteo...
2                           case static amsdu aggreg wlan
3       gaiaeso survey inner disk intermediateag open ...
4       witnessfunct versu interpretationfunct secreci...
                              ...                        
8984    support mixeddatatyp matrix multipl within bli...
8985                          axiomat basi blackwel optim
8986    genevi interact visual tool combin crossdiscip...
8987    quantifi causal effect speed camera road traff...
8988                                   cubemag label grid
Name: TITLE, Length: 8989, dtype: object

for feature in target_features:
    sns.countplot(x=feature,data=train_df)
    plt.xlabel(feature)
    plt.ylabel('count')
    plt.show()
    

for feature in target_features:
    sns.countplot(x=feature,data=train_df[train_df['Quantitative Biology']==1])
    plt.xlabel(feature)
    plt.ylabel('count')
    plt.show()

for feature in target_features:
    sns.countplot(x=feature,data=train_df[train_df['Quantitative Finance']==1])
    plt.xlabel(feature)
    plt.ylabel('count')
    plt.show()

sns.heatmap(train_df[2:8].corr(),annot=True)

In [20]:
l = []
count = 0
for feature in target_features:
    target_feature = target_features.copy()
    target_feature.remove(feature)
    positive_df = train_df[train_df[feature]==1]
    length=len(positive_df)
    #extra_positive_df = train_df[train_df[feature]==1].sample(int(length),random_state=0,replace=True)
    positive_df = pd.concat([positive_df,positive_df],axis=0)
    if count == 4 or count == 5:
        negative_df = train_df[train_df[feature]==0].sample(int(3*length),random_state=0)
    else:
        negative_df = train_df[train_df[feature]==0].sample(int(2*length),random_state=0,replace=True)
    df = pd.concat([positive_df,negative_df],axis=0)
    df.drop(target_feature,axis=1,inplace=True)
    l.append(df)
    count+=1
    
    
    
    

In [21]:
test_df

,TITLE,ABSTRACT,word_count_ABSTRACT,word_count_TITLE,avg_word_ABSTRACT,avg_word_TITLE,stopwords_ABSTRACT,stopwords_TITLE,numerics_ABSTRACT,numerics_TITLE,non_alphanumerics_ABSTRACT,non_alphanumerics_TITLE,upper_case_ABSTRACT,upper_case_TITLE
0,closedform marginal likelihood gammapoisson ma...,present novel understandings gammapoisson gap ...,89,7,6.218750,9.000000,31,1,0,0,0,0,0,0
1,laboratory midir spectra equilibrated igneous ...,meteorites contain minerals solar system aster...,125,14,5.844444,7.214286,41,4,0,0,0,0,0,0
2,case static amsdu aggregation wlans,frame aggregation mechanism multiple frames co...,118,7,5.563492,5.142857,43,1,0,0,0,0,7,1
3,gaiaeso survey inner disk intermediateage open...,milky way open clusters diverse terms age chem...,248,11,5.332090,6.000000,88,1,6,1,1,0,13,1
4,witnessfunctions versus interpretationfunction...,proving cryptographic protocol correct secrecy...,111,11,5.529412,8.636364,53,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8984,supporting mixeddatatype matrix multiplication...,approach problem implementing mixeddatatype su...,183,8,6.040201,8.250000,68,1,1,0,0,0,7,1
8985,axiomatic basis blackwell optimality,theory markov decision processes mdps blackwel...,45,6,5.478261,6.333333,14,1,0,0,0,0,0,0
8986,genevis interactive visualization tool combini...,genevis webbased tool visualize complementary ...,87,12,6.043011,7.333333,30,1,0,0,0,0,0,0
8987,quantifying causal effect speed cameras road t...,paper quantifies effect speed cameras road tra...,147,18,5.805031,5.944444,45,4,0,0,0,0,9,0


In [22]:
target_features

['Computer Science',
 'Physics',
 'Mathematics',
 'Statistics',
 'Quantitative Biology',
 'Quantitative Finance']

num = 5

train_df = l[num]

train_df

target = train_df[target_features[num]]

target

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

CV = CountVectorizer(max_features=1500,ngram_range=(1,2))

train_df0 = CV.fit_transform(train_df['ABSTRACT']).toarray()
test_df0 = CV.transform(test_df['ABSTRACT']).toarray()

train_df1 = CV.fit_transform(train_df['TITLE']).toarray()
test_df1 = CV.transform(test_df['TITLE']).toarray()

train_df0

train_df0 = pd.DataFrame(train_df0)
train_df1 = pd.DataFrame(train_df1,columns=list(range(1500,3000)))

test_df0 = pd.DataFrame(test_df0)
test_df1 = pd.DataFrame(test_df1,columns=list(range(1500,3000)))

train_bog_df = pd.concat([train_df0,train_df1],axis=1)
test_bog_df = pd.concat([test_df0,test_df1],axis=1)

train_bog_df

test_bog_df

train_df.drop(['TITLE','ABSTRACT',target_features[num]],axis=1,inplace=True)

test_df.drop(['TITLE','ABSTRACT'],axis=1,inplace=True)

train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

train_df = pd.concat([train_df,train_bog_df],axis=1)

test_df = pd.concat([test_df,test_bog_df],axis=1)

train_df

test_df

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X_train,X_test,y_train,y_test = train_test_split(train_df,target,test_size=0.2,random_state=0)

model = MultinomialNB(alpha=0.1)

model.fit(X_train,y_train)

y_predict = model.predict(X_test)

y_proba = model.predict_proba(X_test)

from sklearn.metrics import confusion_matrix,accuracy_score

confusion_matrix(y_test,y_predict)

accuracy_score(y_test,y_predict)

y_predict = model.predict(test_df)

pd.Series(y_predict).value_counts()

In [23]:
test_df_copy = test_df.copy()

In [24]:
df_predict = []
for i,df in enumerate(l):
    print(i)
        
    test_df = test_df_copy.copy()
    train_df = df
    target = train_df[target_features[i]]
    
    
    
    from sklearn.feature_extraction.text import CountVectorizer
    

    CV = CountVectorizer(max_features=7500,ngram_range=(1,3))
    
    train_df0 = CV.fit_transform(train_df['TITLE']).toarray()
    test_df0 = CV.transform(test_df['TITLE']).toarray()
   
    
    CV = CountVectorizer(max_features=14000,ngram_range=(1,3))

    train_df1 = CV.fit_transform(train_df['ABSTRACT']).toarray()
    test_df1 = CV.transform(test_df['ABSTRACT']).toarray()
    
    

    train_df0 = pd.DataFrame(train_df0)
    train_df1 = pd.DataFrame(train_df1,columns=list(range(7500,21500)))

    test_df0 = pd.DataFrame(test_df0)
    test_df1 = pd.DataFrame(test_df1,columns=list(range(7500,21500)))

    train_bog_df = pd.concat([train_df0,train_df1],axis=1)
    del train_df0,train_df1
    test_bog_df = pd.concat([test_df0,test_df1],axis=1)
    del test_df0,test_df1
    train_df.drop(['TITLE','ABSTRACT',target_features[i]],axis=1,inplace=True)
    test_df.drop(['TITLE','ABSTRACT'],axis=1,inplace=True)


    train_df.reset_index(drop=True,inplace=True)
    test_df.reset_index(drop=True,inplace=True)

    train_df = pd.concat([train_df,train_bog_df],axis=1)
    del train_bog_df
    test_df = pd.concat([test_df,test_bog_df],axis=1)
    del test_bog_df
    from sklearn.naive_bayes import MultinomialNB

    model = MultinomialNB(alpha=1)

    model.fit(train_df,target)

    y_predict = model.predict(test_df)
    del model
    df_predict.append(pd.Series(y_predict))


0
1
2
3
4
5


In [25]:
df_full_predict = pd.DataFrame(df_predict).T

In [26]:
df_full_predict.columns = target_features

In [27]:
df_full_predict

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,1,0,0
1,0,1,0,0,0,0
2,1,0,0,0,0,0
3,0,1,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
8984,1,0,0,0,0,0
8985,1,0,1,0,0,0
8986,1,0,0,1,1,0
8987,1,0,0,1,0,0


In [28]:
df_full_predict['Quantitative Finance'].value_counts()

0    8889
1     100
Name: Quantitative Finance, dtype: int64

In [29]:
sub = pd.read_csv('sample_submission_UVKGLZE.csv')

In [30]:
sub.drop(target_features,axis=1,inplace=True)

In [31]:
sub = pd.concat([sub,df_full_predict],axis=1)

In [32]:
sub.to_csv('submission.csv',index=False)